In [9]:
## 模型与剖分

import numpy as np
from numpy.linalg import solve
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

from fealpy.decorator import cartesian
from fealpy.mesh import TriangleMesh
from fealpy.functionspace import LagrangeFiniteElementSpace
from fealpy.boundarycondition import DirichletBC

class PDE():
    def __init__(self, mu=1, lam=1):
        self.mu  = mu
        self.lam = lam
        
    def domain(self):
        return [0, 1, 0, 1]
    
    def init_mesh(self, n=1, meshtype='tri'):
        node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
        cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
        mesh = TriangleMesh(node, cell)
        mesh.uniform_refine(n)
        return mesh
    
    @cartesian
    def source(self, p):
        x   = p[..., 0]
        y   = p[..., 1]
        mu  = self.mu
        lam = self.lam
        
        sin = np.sin
        cos = np.cos
        val = np.zeros(p.shape, dtype=np.float64)
        
        val[..., 0] = -((2 * mu + lam) * y * (y - 1) * (2 * cos(x) - (x - 1) * sin(x))
                        + (mu + lam) * (2 * x - 1) * (sin(y) + (y - 1) * cos(y)) 
                        + 2 * mu * (x -1) * sin(x))
        val[..., 1] = -((2 * mu + lam) * x * (x - 1) * (2 * cos(y) - (y - 1) * sin(y))
                        + (mu + lam) * (2 * y - 1) * (sin(x) + (x - 1) * cos(x))
                        + 2 * mu * (y - 1) * sin(y))

        #val[..., 0] = -(2 * (2 * mu + lam) * y * (y - 1) 
        #                + (mu + lam) * (2 * x - 1) * (2 * y - 1)
        #                + 2 * mu * x * (x - 1))
        #val[..., 1] = -(2 * (2 * mu + lam) * x * (x - 1)
        #                + (mu + lam) * (2 * x - 1) * (2 * y - 1)
        #                + 2 * mu * y * (y - 1))
        return val
    
    def dirichlet(self, p):
        var = np.zeros_like(p)
        return var
    
    def solution(self, p):
        x = p[..., 0]
        y = p[..., 1]
        
        val = np.zeros(p.shape, dtype=np.float64)
        
        val[..., 0] = y * (x - 1) * (y - 1) * np.sin(x)
        val[..., 1] = x * (x - 1) * (y - 1) * np.sin(y)
        
        #val[..., 0] = x * (x - 1) * y * (y - 1)
        #val[..., 1] = x * (x - 1) * y * (y - 1)
        return val
        
    def is_dirichlet_boundary(self, p):
        x = p[..., 0]
        y = p[..., 1]
        flag = np.max(np.abs(y)) < 1e-13 
        return flag
    
def error(u, uh):
    e = u - uh
    emax = np.max(np.abs(e))
    return emax

def error_rel(uh1, uh2):
    val = uh2[0:uh1.shape[0]] - uh1
    return np.max(np.abs(val))

def print_error(Lam, H, E, E_rel):
    for i in range(len(Lam)):
        print("lam= ", Lam[i])
        for j in range(n):
            print("h= ", H[j])
            print("e=", E[i][j])
            print("e_rel= ", E_rel[i,j])
            print()
        print()
    
def print_P(Lam, P):
    for i in range(len(Lam)):
        print("lam= ", Lam[i])
        print("p= ", P[i])
        print()

n   = 7  #剖分次数
n   = n + 1 
Lam = [1,10,1e3,1e4,1e5]

H     = np.zeros(n)                              #步长
P     = np.zeros(len(Lam))                       #误差阶
E     = np.zeros((len(Lam),n), dtype=np.float64) #每个lambda(行)对应的误差(列)
E_rel = np.zeros_like(E)

for i in range(len(Lam)):
    pde = PDE(1,Lam[i])
    for j in range(n):
        mesh = pde.init_mesh(j)
        NN = mesh.number_of_nodes()
        NC = mesh.number_of_cells()
        #[NC,2] 剖分点及其编号(下标)
        node = mesh.entity('node')
        #print("node= ", node)
        #[NC,3] 剖分区间及其端点编号
        cell = mesh.entity('cell')
        #print("cell= ", cell)
        #print("node[cell]= ", node[cell])
        #[NC] 每个单元的面积
        cm = mesh.entity_measure()


        ## 刚度矩阵

        #cr基函数的编号及所在点其坐标
        #print("node[cell]= ", node[cell])
        cr_node_cell = np.sum(node[cell], axis=1)
        cr_node_cell = np.broadcast_to(cr_node_cell[:, None, :], shape=node[cell].shape)
        cr_node_cell = (cr_node_cell - node[cell]) / 2 
        #print("cr_node_cell= ", cr_node_cell)
        
        ##求解CR元导数
        cr_node_cell_A = np.ones((NC, 3, 3), dtype=np.float64)
        #求解CR元导数的系数矩阵
        cr_node_cell_A[:, :, 0:2] = cr_node_cell
        #print("cr_node_cell_A= ", cr_node_cell_A)
        #用于求解CR元的值
        cr_glam_x_y_pre = np.zeros((NC, 3, 3), dtype=np.float64)
        for k in range(NC):
            cr_glam_x_y_pre[k, :, :] = solve(cr_node_cell_A[k, :, :], np.diag(np.ones(3)))
        #print("cr_glam_x_y= ", cr_glam_x_y)
        #[NC,3,3]
        cr_glam_x_y = np.copy(cr_glam_x_y_pre)
        cr_glam_x_y = cr_glam_x_y[:, 0:2, :]
        #print("cr_glam_x_y= ", cr_glam_x_y)
        cr_glam_x_y = cr_glam_x_y.transpose((0,2,1))
        #print("cr_glam_x_y= ", cr_glam_x_y)
        
    
        ##[NC,3,2] 每个单元上线性基函数(三个)对x、y的偏导数
        #glam_x_y    = mesh.grad_lambda()
        glam_x_y = cr_glam_x_y
        #print("glam_x_y= ", glam_x_y)
        glam_dim4 = np.broadcast_to(glam_x_y[:, :, None, :], shape=(NC,3,2,2)).copy()
        #print("glam_dim4= ", glam_dim4)
        glam_dim5 = np.broadcast_to(glam_dim4[:, :, :, None, :], shape=(NC,3,2,2,2)).copy()
        #print("glam_dim5= ", glam_dim5)
        glam_dim5[:, :, 0, 1, :] = 0 # [NC,3(三个节点),2(每个点两个基函数),2,2(后两个指标指基函数的 grad)]
        glam_dim5[:, :, 1, 0, :] = 0 # x方向上的基函数第二个分量为0，y方向上的基函数第一个分量为0
        #print("glam_dim5= ", glam_dim5)

        # [NC,3,2]
        glam_div = np.einsum("cnmij->cnm", glam_dim5)
        # [NC,6(每个基函数的 div)]
        glam_div = glam_div.reshape(NC,6)
        #print("glam_sum= ", glam_sum)

        glam_v = glam_dim5.reshape(NC,6,2,2)
        #print("glam_v= ", glam_v)
        glam_t = glam_dim5.swapaxes(3,4)
        glam_dim5 = glam_dim5 + glam_t
        #print("glam_dim5= ", glam_dim5)
        glam_dim4 = glam_dim5.reshape(NC,6,2,2)  #后两个指标指每个基函数的 grad + grad^t
        #print("glam_dim4= ", glam_dim4)

        # [NC,6,6]
        S = np.einsum("cnij, cmij, c->cnm", glam_dim4, glam_v, cm)
        #[NC,6,6] 单元刚度矩阵第二部分
        M = np.einsum("cn, cm, c->cnm", glam_div, glam_div, cm)

        cell_x_y = np.broadcast_to(cell[:,:,None], shape=(NC, 3, 2)).copy()
        cell_x_y[:,:,0] = 2 * cell_x_y[:,:,0]       #[NC,3] 三个节点x方向上基函数在总刚度矩阵的位置
        cell_x_y[:,:,1] = 2 * cell_x_y[:,:,1] + 1   #[NC,3] 三个节点y方向上基函数在总刚度矩阵的位置
        #print("cell= ", cell)
        #print("cell_x_y= ", cell_x_y)
        cell_x_y = cell_x_y.reshape(NC, 6)
        #print("cell_x_y= ", cell_x_y)
        I = np.broadcast_to(cell_x_y[:, :, None], shape=S.shape)
        J = np.broadcast_to(cell_x_y[:, None, :], shape=S.shape)
        #print("I= ", I)
        #print("J= ", J)

        S = csr_matrix((S.flat, (I.flat, J.flat)), shape=(2 * NN,2 * NN))
        M = csr_matrix((M.flat, (I.flat, J.flat)), shape=(2 * NN,2 * NN))
        A = pde.mu * S + pde.lam * M
        #print("A= ", A.toarray())


        ##载荷向量

        from fealpy.quadrature import TriangleQuadrature

        NQ = 3
        Q = int(NQ * (NQ + 1) / 2) # 内积分点数
        qf = TriangleQuadrature(NQ)

        #bcs [Q,3]; WS[Q,]
        bcs,ws = qf.get_quadrature_points_and_weights()
        #print("bcs= ", bcs)
        #phi [Q,3,2]
        phi = np.broadcast_to(bcs[:, :, None], shape=[Q,3,2])
        #print("phi= ", phi.shape)

        ##求CR元在各区间的值
        cr_phi_x_y  = np.zeros((Q, 3, 2, 2), dtype=np.float64)
        node_cell_A = np.ones((NC,3,3), dtype=np.float64)
        node_cell_A[:, :, 0:2] = node[cell]
        #print("node_cell_A= ", node_cell_A)
        #[NC,3(3个点),3(3个基函数)]CR元在原剖分节点的值
        cr_node_x_y = np.einsum("cjk, ckm -> cjm", node_cell_A, cr_glam_x_y_pre)
        #print("cr_glam_x_y_pre= ", cr_glam_x_y_pre)
        #print("cr_node_x_y= ", cr_node_x_y)
        
        #[Q,3,2(每个内积分节点两个基函数),2(每个基函数两个分量)]
        phi_x_y = np.zeros((Q, 3, 2, 2), dtype=np.float64) 
        phi_x_y[:, :, :, 0] = phi
        phi_x_y[:, :, :, 1] = phi
        phi_x_y[:, :, 0, 1] = 0
        phi_x_y[:, :, 1, 0] = 0
        phi_x_y = phi_x_y.reshape(Q,6,2)
        #print("phi_x_y= ", phi_x_y)

        # node[cell] [NC,3(三个节点),2(每个节点的 x, y 坐标)]
        # ps [Q,NC,2]
        # 内积分点标准单元坐标转换为真实坐标
        ps = np.einsum('qi, cim->qcm', bcs, node[cell]) 
        #print("ps= ", ps)
        
        #求CR元在内积分节点的值
        cr_ps = np.ones((Q,NC,3))
        cr_ps[:, :, 0:2] = ps
        #print("cr_ps= ", cr_ps)
        #[Q,NC,3]
        cr_ps_val = np.einsum("qni, nik -> qnk", cr_ps, cr_glam_x_y_pre)
        #print("cr_ps_val= ", cr_ps_val)
        cr_ps_x_y = np.zeros((Q, NC, 3, 2, 2), dtype=np.float64)
        cr_ps_x_y[:, :, :, 0, 0] = cr_ps_val
        cr_ps_x_y[:, :, :, 1, 1] = cr_ps_val
        #print("cr_ps_x_y= ", cr_ps_x_y)
        cr_ps_x_y = cr_ps_x_y.reshape(Q, NC, 6, 2)
        
        # val [Q,NC,2] 每个内积分的右端项的值
        val = pde.source(ps)
        #print("val= ", val)
        #bb [NC,6]
        bb = np.einsum('q, qcj, qij, c -> ci', ws, val, phi_x_y, cm)
        bb = np.einsum("q, qcj, qcij, qij, c -> ci", ws, val, cr_ps_x_y, phi_x_y, cm)
        #print("cell= ", cell)
        #print("bb= ", bb)
        #print("cel_x_y= ", cell_x_y)

        F = np.zeros(2 * NN)
        np.add.at(F, cell_x_y, bb)
        #print("F= ", F)


        ## 求解

        isBdNode = mesh.ds.boundary_node_flag()
        isInterNode = ~isBdNode
        #print("isInterNode= ", isInterNode)
        isInterNode = np.broadcast_to(isInterNode[:, None], shape=(NN, 2))
        isInterNode = isInterNode.reshape(2 * NN)
        #print("isInterNode= ", isInterNode)

        uh = np.zeros((2 * NN), dtype=np.float64)
        uh[isInterNode] = spsolve(A[:, isInterNode][isInterNode], F[isInterNode])
        #print("uh= ", uh)
        uh = uh.reshape(NN, 2)
        #print("uh= ", uh)
        #print("F[isInterNode]= ", F[isInterNode])
        u = pde.solution(node)
        H[j] = cm[0]
        E[i][j] = error(u, uh)
        if j > 0:
            E_rel[i][j] = error_rel(uh_old, uh)
        uh_old = uh

for i in range(len(Lam)):
    fig = plt.figure()
    plt.plot(np.log(H[2:n]), np.log(E_rel[i][2:n]))
    plt.title("lam={}".format(Lam[i]))
    plt.xlabel("log(h)")
    plt.ylabel("log(e)")
    plt.savefig(fname="../image/tmp/CRFem/elasticityCRFemLam_{}.png"
                .format(Lam[i]))
    plt.close(fig)
    
    f = np.polyfit(np.log(H[2:n]), np.log(E_rel[i][2:n]),1)
    P[i] = f[0]

#print_error(Lam, H, E, E_rel)
print_P(Lam, P)

lam=  1
p=  1.0248639295934008

lam=  10
p=  1.02991937963952

lam=  1000.0
p=  0.8151612863649255

lam=  10000.0
p=  0.5796258396881044

lam=  100000.0
p=  0.34245499243405275

